In [1]:
import requests
# from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import pickle
import time

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from collections import defaultdict

import pipeline as p

%load_ext autoreload
%autoreload 2

import datetime as dt

In [2]:
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

chromedriver = "/Users/kendra/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url_base = 'https://www.pro-football-reference.com'

### Define year

In [3]:
year = 2018

### Load existing data

In [18]:
team_dict = p.open_pkl('TeamDicts/team_dictionary.pkl')

# may need to start from scratch -- sometimes boxscore hrefs change during season
#     bxsc_html = defaultdict(list)    # [href] = html -- also tracks whether a bxsc href has been visited
bxsc_html = p.open_pkl(f'Data/BxscHtmls/bxsc_html_{str(year)}.pkl')

### Update data

In [54]:
# Season Data from Team-Year webpages

driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)

season_html = {}  
bxsc_refs = {}                   # [team-year] = list of hrefs for boxscores

for tm_list in team_dict.values():
    team, url = tm_list[1], tm_list[2]
    tm_yr = team +'-' + str(year)

    # get html of season data table
    html = p.get_season_data(url, year, driver)

    # if successfully returned data:
    if html:
        # add html to season_html dictionary
        season_html[tm_yr] = html
        
        # generate list of individaul game boxscore URLs, add them to bxsc_refs dictionary
        href_list = p.get_href_list(html)
        bxsc_refs[tm_yr] = href_list

        # get & add bxsc html's to bxsc_html dict
        for a in href_list:
            href = a.attrs['href']
            if href not in bxsc_html.keys():
                bxsc_html[href] = p.get_bxsc_data(href, driver)


driver.close()

print('Done')

p.pkl_this(f'Data/season_html_{str(year)}.pkl', season_html)
p.pkl_this(f'Data/bxsc_html_{str(year)}.pkl', bxsc_html)

Loading page for nwe 2018
Page loaded & data found!
Loading page for tam 2018
Page loaded & data found!
Loading page for chi 2018
Page loaded & data found!
Skipping page https://www.pro-football-reference.com/boxscores/201901060chi.htm
Loading page for sfo 2018
Page loaded & data found!
Loading page for htx 2018
Page loaded & data found!
Skipping page https://www.pro-football-reference.com/boxscores/201901050htx.htm
Loading page for rai 2018
Page loaded & data found!
Loading page for nyg 2018
Page loaded & data found!
Loading page for kan 2018
Page loaded & data found!
Loading page for nor 2018
Page loaded & data found!
Loading page for crd 2018
Page loaded & data found!
Loading page for rav 2018
Page loaded & data found!
Skipping page https://www.pro-football-reference.com/boxscores/201901060rav.htm
Loading page for det 2018
Page loaded & data found!
Loading page for car 2018
Page loaded & data found!
Loading page for sea 2018
Page loaded & data found!
Skipping page https://www.pro-fo

In [57]:
p.pkl_this(f'Data/season_html_{str(year)}.pkl', season_html)
p.pkl_this(f'Data/bxsc_html_{str(year)}.pkl', bxsc_html)

In [90]:
# to update Game Data from Boxscores webpages separately

if 1 == 1:
    driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)

    for href, html in bxsc_html.items():
        if html == None:
            print('Updating ' + href)
            bxsc_html[href] = p.get_bxsc_data(href, driver)
    
    print('Done')
    driver.close()

    p.pkl_this(f'Data/bxsc_html_{str(year)}.pkl', bxsc_html)

Updating /boxscores/201812090rai.htm
Updating /boxscores/201811040rav.htm
Updating /boxscores/201810140dal.htm
Updating /boxscores/201811080pit.htm
Updating /boxscores/201811250buf.htm
Updating /boxscores/201810140nyj.htm
Updating /boxscores/201810210nyj.htm
Updating /boxscores/201811040buf.htm
Updating /boxscores/201811110rai.htm
Updating /boxscores/201811180clt.htm
Updating /boxscores/201811040nwe.htm
Updating /boxscores/201811180atl.htm
Updating /boxscores/201811180chi.htm
Updating /boxscores/201812160buf.htm
Updating /boxscores/201812240rai.htm
Updating /boxscores/201812020atl.htm
Updating /boxscores/201812300nwe.htm
Updating /boxscores/201812020oti.htm
Updating /boxscores/201811180sdg.htm
Updating /boxscores/201810210kan.htm
Updating /boxscores/201811040min.htm
Updating /boxscores/201811250phi.htm
Updating /boxscores/201810280pit.htm
Updating /boxscores/201810250htx.htm
Updating /boxscores/201810280jax.htm
Updating /boxscores/201811110cle.htm
Updating /boxscores/201812160ram.htm
U

In [91]:
# to clean up bxsc_html

if 1 == 1:
    del_list = []
    for href, html in bxsc_html.items():
        if html == None:
            del_list.append(href)
    

In [92]:
del_list

['/boxscores/201812230sdg.htm', '/boxscores/201812230oti.htm']

In [93]:
for href in del_list:
    del bxsc_html[href]

In [94]:
len(bxsc_html)

254

In [95]:
p.pkl_this(f'Data/bxsc_html_{str(year)}.pkl', bxsc_html)

#### Fix these:

In [97]:
driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)

In [98]:
href = '/boxscores/201812220sdg.htm'
bxsc_html[href] = p.get_bxsc_data(href, driver)

In [99]:
# driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)
href = '/boxscores/201812220oti.htm'
bxsc_html[href] = p.get_bxsc_data(href, driver)

In [100]:
len(bxsc_html)

256

In [101]:
p.pkl_this(f'Data/bxsc_html_{str(year)}.pkl', bxsc_html)

In [102]:
driver.close()

### Clean Season Data

In [58]:
url_team_dict = p.open_pkl('TeamDicts/url_team_dict.pkl')
file_dir = 'Data/SeasonHtmls/'
# os.chdir('Data/SeasonHtmls/')
# count = 0

master_df = pd.DataFrame()
merge_dict = defaultdict(list)

# for f in os.listdir():
for f in os.listdir(file_dir):
    
    if f[0] != '.' and int(f[4:8]) == year:
        team = url_team_dict[f[:3].lower()][2]

        if team not in merge_dict[year]:
#             print('Processing ' + team, year)
            html = p.open_pkl(file_dir + f)
            try:
                # covert to df & clean 
                df = p.clean_season_data(html, team, year)

                # append to master df
                master_df = master_df.append(df, ignore_index=True, sort=True)

                # add entry to tsdf_merge_track
                merge_dict[year].append(team)

            except ValueError as e: 
                print('Error cleaning ' + team, year, e)
                p.log_this('Error cleaning ' + team, year, e)


print('Done')   

/Users/kendra/anaconda3/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Done


In [103]:
master_df.head()

,1stD-O,Date,Day,DefTO,Location,OT,Opp,PassY,PtsOpp,PtsTm,Record,Result,RushY,TO,Team,Time,TotY,Week,Year
0,26.0,2018-09-09,Sun,NaN,NaN,NaN,Tampa Bay Buccaneers,432.0,48.0,40.0,0-1,L,43.0,2.0,NOR,13:00:00,475.0,1,2018
1,18.0,2018-09-16,Sun,1.0,NaN,NaN,Cleveland Browns,213.0,18.0,21.0,1-1,W,62.0,2.0,NOR,13:00:00,275.0,2,2018
2,32.0,2018-09-23,Sun,NaN,@,OT,Atlanta Falcons,391.0,37.0,43.0,2-1,W,143.0,NaN,NOR,13:00:00,534.0,3,2018
3,25.0,2018-09-30,Sun,2.0,@,NaN,New York Giants,219.0,18.0,33.0,3-1,W,170.0,NaN,NOR,16:25:00,389.0,4,2018
4,27.0,2018-10-08,Mon,2.0,NaN,NaN,Washington Redskins,349.0,19.0,43.0,4-1,W,98.0,1.0,NOR,20:15:00,447.0,5,2018


In [12]:
# 520 games in regular season; 531 including play-offs
print (len(master_df))
len(merge_dict[year]) == 32

520


True

In [60]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 19 columns):
1stD-O      512 non-null float64
Date        520 non-null datetime64[ns]
Day         520 non-null object
DefTO       369 non-null float64
Location    260 non-null object
OT          30 non-null object
Opp         520 non-null object
PassY       512 non-null float64
PtsOpp      512 non-null float64
PtsTm       512 non-null float64
Record      512 non-null object
Result      512 non-null object
RushY       512 non-null float64
TO          369 non-null float64
Team        520 non-null object
Time        520 non-null object
TotY        512 non-null float64
Week        520 non-null int64
Year        520 non-null int64
dtypes: datetime64[ns](1), float64(8), int64(2), object(8)
memory usage: 77.3+ KB


In [61]:
master_df.loc[0,'Time']

datetime.time(13, 0)

In [62]:
p.pkl_this(f'../master_df_{str(year)}.pkl', master_df)

### Clean Boxscore Data

In [104]:
count = 0

bxsc_df = pd.DataFrame()

for key, val in bxsc_html.items():
    week_date = dt.datetime.strptime(key.split("/")[2][:8], '%Y%m%d')

    if week_date < dt.datetime.now():
#         print(week_date)
        team = url_team_dict[key[-7:-4]][2]
#         print(team)
        week = master_df[(master_df['Date'] == week_date) & (master_df['Team'] == team)]['Week'].values[0]
        try:
            df = p.clean_bxsc_data(val, week, year)

            # Append boxscore df to bxsc_df
            bxsc_df = bxsc_df.append(df)

            count += 1
            if count%20 == 0:
                print(count)
        except:
            print(f'Error: {week_date} & {team}')
    

print ('Done!')

20
40
60
80
100
120
140
160
180
200
220
240
Done!


In [106]:
p.pkl_this(f'Data/bxsc_df_{str(year)}.pkl', bxsc_df)

### Merge bxsc_df with master_df

In [107]:
df = pd.merge(master_df, bxsc_df, how='left', on=['Team', 'Week','Year'])

In [109]:
df.head()

,1stD-O,Date,Day,DefTO,Location,OT,Opp,PassY,PtsOpp,PtsTm,...,PassAtt,PassTDs,INT,SacksO,Fumbles,Penalies,PenY,3rdDConv,3rdDAtt,4thDAtt
0,26.0,2018-09-09,Sun,NaN,NaN,NaN,Tampa Bay Buccaneers,432.0,48.0,40.0,...,45.0,3.0,0.0,1.0,2.0,8.0,77.0,4.0,9.0,1.0
1,18.0,2018-09-16,Sun,1.0,NaN,NaN,Cleveland Browns,213.0,18.0,21.0,...,35.0,2.0,0.0,3.0,2.0,4.0,53.0,3.0,12.0,0.0
2,32.0,2018-09-23,Sun,NaN,@,OT,Atlanta Falcons,391.0,37.0,43.0,...,49.0,3.0,0.0,1.0,1.0,11.0,120.0,7.0,14.0,1.0
3,25.0,2018-09-30,Sun,2.0,@,NaN,New York Giants,219.0,18.0,33.0,...,34.0,0.0,0.0,1.0,1.0,5.0,51.0,5.0,13.0,1.0
4,27.0,2018-10-08,Mon,2.0,NaN,NaN,Washington Redskins,349.0,19.0,43.0,...,29.0,3.0,0.0,2.0,2.0,4.0,45.0,5.0,9.0,0.0


In [108]:
len(df), len(master_df)

(520, 520)

In [110]:
df.columns

Index(['1stD-O', 'Date', 'Day', 'DefTO', 'Location', 'OT', 'Opp', 'PassY',
       'PtsOpp', 'PtsTm', 'Record', 'Result', 'RushY', 'TO', 'Team', 'Time',
       'TotY', 'Week', 'Year', 'Time of Possession', 'RushAtt', 'RushTDs',
       'PassCmp', 'PassAtt', 'PassTDs', 'INT', 'SacksO', 'Fumbles', 'Penalies',
       'PenY', '3rdDConv', '3rdDAtt', '4thDAtt'],
      dtype='object')

In [111]:
df.rename(columns={'1stD-O':'1stD'}, inplace=True)

In [112]:
p.pkl_this(f'Data/merged_df_{str(year)}.pkl', df)

### Merge with other season's merged data

In [126]:
merged_df = p.open_pkl('Data/merged_df.pkl')

Old merged_df did not have Time column converted. Do so here, before merging:

In [129]:
merged_df.loc[0,'Time']

'1:03PM ET'

In [131]:
merged_df['Time'] = pd.to_datetime(merged_df['Time'], utc=False).dt.time

/Users/kendra/anaconda3/lib/python3.6/site-packages/dateutil/parser/_parser.py:1204: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [132]:
merged_df.loc[0,'Time']

datetime.time(13, 3)

Check that columns match

In [123]:
sorted(merged_df.columns) == sorted (df.columns)

True

In [133]:
new_merge = merged_df.append(df, sort=True)

In [119]:
new_merge.head()

,1stD,3rdDAtt,3rdDConv,4thDAtt,Date,Day,DefTO,Fumbles,INT,Location,...,RushTDs,RushY,SacksO,TO,Team,Time,Time of Possession,TotY,Week,Year
0,20.0,17.0,5.0,4.0,2014-09-07,Sun,3.0,3.0,0.0,@,...,1.0,89.0,4.0,2.0,NWE,1:03PM ET,29:33,315.0,1,2014
1,16.0,14.0,5.0,1.0,2014-09-14,Sun,4.0,1.0,0.0,@,...,1.0,150.0,1.0,NaN,NWE,1:05PM ET,31:34,292.0,2,2014
2,21.0,18.0,9.0,1.0,2014-09-21,Sun,1.0,1.0,0.0,NaN,...,0.0,76.0,2.0,NaN,NWE,1:02PM ET,31:35,297.0,3,2014
3,13.0,9.0,2.0,0.0,2014-09-29,Mon,NaN,2.0,2.0,@,...,0.0,75.0,3.0,3.0,NWE,8:31PM ET,23:33,290.0,4,2014
4,30.0,16.0,6.0,1.0,2014-10-05,Sun,3.0,1.0,0.0,NaN,...,1.0,220.0,1.0,NaN,NWE,8:43PM ET,38:56,505.0,5,2014


In [137]:
new_merge.reset_index(drop=True, inplace=True)

In [138]:
new_merge.loc[0]

1stD                                 20.0
3rdDAtt                              17.0
3rdDConv                              5.0
4thDAtt                               4.0
Date                  2014-09-07 00:00:00
Day                                   Sun
DefTO                                 3.0
Fumbles                               3.0
INT                                   0.0
Location                                @
OT                                    NaN
Opp                        Miami Dolphins
PassAtt                              56.0
PassCmp                              29.0
PassTDs                               1.0
PassY                               226.0
PenY                                100.0
Penalies                              9.0
PtsOpp                               33.0
PtsTm                                20.0
Record                                0-1
Result                                  L
RushAtt                              20.0
RushTDs                           

In [135]:
len(merged_df), len(new_merge)

(5874, 6394)

In [134]:
p.pkl_this('Data/merged_df.pkl', new_merge)

### Elo Data from FiveThirtyEight

In [71]:
elo_url = 'https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv'
elo_df = pd.read_csv(elo_url)

In [72]:
elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2
0,1920-09-26,1920,0,NaN,RII,STP,1.5k,1.3k,824.7m,175.3m,1.5k,1.3k,48.0,0.0
1,1920-10-03,1920,0,NaN,BFF,WBU,1.5k,1.3k,802.0m,198.0m,1.5k,1.3k,32.0,6.0
2,1920-10-03,1920,0,NaN,CBD,PTQ,1.5k,1.3k,825.3m,174.7m,1.5k,1.3k,48.0,0.0
3,1920-10-03,1920,0,NaN,CHI,MUT,1.4k,1.3k,683.0m,317.0m,1.4k,1.3k,20.0,0.0
4,1920-10-03,1920,0,NaN,RII,MUN,1.5k,1.5k,644.2m,355.8m,1.5k,1.5k,45.0,0.0


In [75]:
elo_df['date'] = pd.to_datetime(elo_df['date'], format='%Y-%m-%d')
elo_df = elo_df[elo_df['date'] >= dt.datetime(2007,8,1)].reset_index(drop=True) # Can drop anything before 2007
p.pkl_this('Data/elo_df.pkl', elo_df)

## *Proceed to '02c_TrailingSeasonAvgs_MergeElo'*